In [1]:
import sys
import os
import time
import numpy as np
import theano
import theano.tensor as T
import lasagne
import pandas as pd

def loaddata():
    
    data=pd.read_csv('table.csv',usecols=(1,2,3,4,5),dtype=np.float32)
    data=np.array(data)
    target=np.log(data[2700::-1,3])
    target=np.diff(target)
    target=np.ndarray.astype((np.sign(np.sign(target)+0.001)+1)/2,dtype=np.int32)
    data=data[2700:0:-1,:]/1000
    data[:,4]=data[:,4]/1000000
    data=data.reshape(2700,1,5)
    train_data=data[0:2500,:,:]
    train_target=target[0:2500]
    val_data=data[2500:2600,:,:]
    val_target=target[2500:2600]
    test_data=data[2600:,:,:]
    test_target=target[2600:]
    return train_data,train_target,val_data,val_target,test_data,test_target

In [8]:
data=loaddata()[0]

In [10]:
N_HIDDEN = 100 #number of hidden nodes in the LSTM
GRAD_CLIP = 200 #Gradient higher than this will be clipped

l_in = lasagne.layers.InputLayer(shape=(None, 1, 5))

l_forward = lasagne.layers.RecurrentLayer(
    l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
    W_in_to_hid=lasagne.init.HeUniform(),
    W_hid_to_hid=lasagne.init.HeUniform(),
    nonlinearity=lasagne.nonlinearities.tanh)

l_out = lasagne.layers.DenseLayer(
    l_forward, num_units=2, nonlinearity=lasagne.nonlinearities.softmax)

target_values = T.ivector('target_output')

hidstate = lasagne.layers.get_output(l_forward)

pred=theano.function([l_in.input_var],hidstate)

a=pred(data[:10,:,:])

print(a)

[[[  9.24049862e-01  -5.33296625e-01  -7.99198390e-01  -3.10890876e-01
     4.70234251e-01   6.51910103e-01  -3.37428109e-01   8.87427121e-01
     9.50558232e-01  -5.60723713e-01  -4.57663605e-01   9.08813724e-01
    -9.25893068e-01  -9.89722003e-01  -8.70361058e-01  -9.21775387e-01
     9.95349324e-01   9.05872804e-02   9.07038402e-01  -9.21153509e-01
    -6.13715355e-01  -9.10266596e-01  -9.11650969e-01  -9.54541327e-01
    -9.82856576e-01  -4.75511297e-02   8.98797806e-01  -9.96679976e-01
    -9.66076048e-01   8.72972307e-01  -6.02525508e-01  -4.69477853e-01
     8.70676488e-01  -2.52190121e-01  -8.70145235e-01   9.56824979e-01
    -8.12234131e-01  -8.56394528e-01  -8.64107889e-01   6.08852144e-01
     8.82868149e-01   9.98527841e-01   6.59473084e-02  -9.86865138e-01
    -8.17329691e-01   9.97024589e-01   8.00221987e-01   1.02485509e-01
    -5.41393677e-01   2.93933304e-01   2.23629876e-02   8.74200787e-01
     9.34458971e-01  -7.51823794e-01   8.08220300e-01  -6.16588705e-01
     6

/Users/ruiqiao/anaconda/lib/python3.4/site-packages/theano/scan_module/scan.py:1019: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


In [11]:
a.shape

(10, 1, 100)